In [5]:
import math
import datetime
import time
import string
import re
from glob import glob

import numpy as np
import pandas as pd
import yfinance as yf
import yahooquery as yq
import yahoo_fin.stock_info as stock_info

from utils import *

In [3]:
all_tickers = get_all_usa_tickers()

In [ ]:
ticker = "PGY"
yf_ticker = yf.Ticker(ticker)

In [3]:
df = yf_ticker.history(period="5d", raise_errors=True)

In [ ]:
ohlcv = pd.read_parquet("ASSETS/ohlcv.parquet")
ohlcv_index_tz = ohlcv.loc["A"].index.tz

parquet_paths = glob("DB/**/ohlcv.parquet")
for parquet_path in parquet_paths:
    end_date = datetime.datetime.strptime(parquet_path.split(os.sep)[-2], "%y%m%d")
    start_date = end_date - datetime.timedelta(days=6)
    start_date = pd.Timestamp(start_date).tz_localize(ohlcv_index_tz)
    end_date = pd.Timestamp(end_date).tz_localize(ohlcv_index_tz)

    df = pd.read_parquet(parquet_path)
    week_ohlcv = ohlcv.loc[(slice(None), slice(start_date, end_date)), :]
    new_ohlcv = []
    for ticker in week_ohlcv.index.get_level_values(0).unique().tolist():
        ticker_ohlcv = week_ohlcv.loc[ticker]
        ticker_ohlcv = week_ohlcv[["Open", "High", "Low", "Close", "Volume"]].loc["A"].T
        ticker_ohlcv.index.name = "Type"
        ticker_ohlcv["Ticker"] = ticker
        ticker_ohlcv = ticker_ohlcv.reset_index().set_index(["Ticker", "Type"])
        ticker_ohlcv.columns = [str(col.date()) for col in ticker_ohlcv.columns]
        new_ohlcv.append(ticker_ohlcv)
    new_ohlcv = pd.concat(new_ohlcv, axis=0)
    new_ohlcv.to_parquet(parquet_path.replace("price", "ohlcv"))